# Policy gradients


In this notebook, we'll build a neural network that can learn to play games through reinforcement learning. More specifically, we'll use Q-learning to train an agent to play a game called [Cart-Pole](https://gym.openai.com/envs/CartPole-v0). In this game, a freely swinging pole is attached to a cart. The cart can move to the left and right, and the goal is to keep the pole upright as long as possible.

![Cart-Pole](assets/cart-pole.jpg)

We can simulate this game using [OpenAI Gym](https://gym.openai.com/). First, let's check out how OpenAI Gym works. Then, we'll get into training an agent to play the Cart-Pole game.

In [1]:
# In this one we should define and detect GPUs for tensorflow
# GPUs or CPU
import tensorflow as tf

# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


>**Note:** Make sure you have OpenAI Gym cloned into the same directory with this notebook. I've included `gym` as a submodule, so you can run `git submodule --init --recursive` to pull the contents into the `gym` repo.

##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import gym

## Create the Cart-Pole game environment
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('MountainCarContinuous-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


We interact with the simulation through `env`. To show the simulation running, you can use `env.render()` to render one frame. Passing in an action as an integer to `env.step` will generate the next step in the simulation.  You can see how many actions are possible from `env.action_space` and to get a random action you can use `env.action_space.sample()`. This is general to all Gym games. In the Cart-Pole game, there are two possible actions, moving the cart left or right. So there are two actions we can take, encoded as 0 and 1.

Run the code below to watch the simulation run.

In [3]:
env.reset()
batch = []
for _ in range(1111):
    #env.render()
    action = env.action_space.sample()
    state, reward, done, info = env.step(action) # take a random action
    batch.append([action, state, reward, done, info])
    #print('state, action, reward, done, info:', state, action, reward, done, info)
    if done:
        env.reset()

To shut the window showing the simulation, use `env.close()`.

In [4]:
# env.close()

If you ran the simulation above, we can look at the rewards:

In [5]:
batch[0], 
batch[0][1].shape, state.shape

((4,), (4,))

In [6]:
import numpy as np
actions = np.array([each[0] for each in batch])
states = np.array([each[1] for each in batch])
rewards = np.array([each[2] for each in batch])
dones = np.array([each[3] for each in batch])
infos = np.array([each[4] for each in batch])

In [7]:
# print(rewards[-20:])
print('shapes:', np.array(rewards).shape, np.array(states).shape, np.array(actions).shape, np.array(dones).shape)
print('dtypes:', np.array(rewards).dtype, np.array(states).dtype, np.array(actions).dtype, np.array(dones).dtype)
print('states:', np.max(np.array(states)), np.min(np.array(states)))
print('actions:', np.max(np.array(actions)), np.min(np.array(actions)))
# print((np.max(np.array(actions)) - np.min(np.array(actions)))+1)
print('rewards:', np.max(np.array(rewards)), np.min(np.array(rewards)))

shapes: (1111,) (1111, 4) (1111,) (1111,)
dtypes: float64 float64 int64 bool
states: 2.440401649885929 -2.9888261858931062
actions: 1 0
rewards: 1.0 1.0


In [8]:
actions[:10]

array([0, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [9]:
rewards[:10]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [10]:
# import numpy as np
def sigmoid(x, derivative=False):
  return x*(1-x) if derivative else 1/(1+np.exp(-x))

In [11]:
sigmoid(np.max(np.array(rewards))), sigmoid(np.min(np.array(rewards)))

(0.7310585786300049, 0.7310585786300049)

In [12]:
print('rewards:', np.max(np.array(rewards))/100, np.min(np.array(rewards))/100)

rewards: 0.01 0.01


The game resets after the pole has fallen past a certain angle. For each frame while the simulation is running, it returns a reward of 1.0. The longer the game runs, the more reward we get. Then, our network's goal is to maximize the reward by keeping the pole vertical. It will do this by moving the cart to the left and the right.

## Q-Network

We train our Q-learning agent using the Bellman Equation:

$$
Q(s, a) = r + \gamma \max{Q(s', a')}
$$

where $s$ is a state, $a$ is an action, and $s'$ is the next state from state $s$ and action $a$.

Before we used this equation to learn values for a Q-_table_. However, for this game there are a huge number of states available. The state has four values: the position and velocity of the cart, and the position and velocity of the pole. These are all real-valued numbers, so ignoring floating point precisions, you practically have infinite states. Instead of using a table then, we'll replace it with a neural network that will approximate the Q-table lookup function.

<img src="assets/deep-q-learning.png" width=450px>

Now, our Q value, $Q(s, a)$ is calculated by passing in a state to the network. The output will be Q-values for each available action, with fully connected hidden layers.

<img src="assets/q-network.png" width=550px>


As I showed before, we can define our targets for training as $\hat{Q}(s,a) = r + \gamma \max{Q(s', a')}$. Then we update the weights by minimizing $(\hat{Q}(s,a) - Q(s,a))^2$. 

For this Cart-Pole game, we have four inputs, one for each value in the state, and two outputs, one for each action. To get $\hat{Q}$, we'll first choose an action, then simulate the game using that action. This will get us the next state, $s'$, and the reward. With that, we can calculate $\hat{Q}$ then pass it back into the $Q$ network to run the optimizer and update the weights.

Below is my implementation of the Q-network. I used two fully connected layers with ReLU activations. Two seems to be good enough, three might be better. Feel free to try it out.

In [13]:
# Data of the model
def model_input(state_size):
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    targetQs = tf.placeholder(tf.float32, [None], name='targetQs')
    reward = tf.placeholder(tf.float32, [], name='reward')
    return states, actions, targetQs, reward

In [14]:
# Generator/Controller: Generating/prediting the actions
def generator(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('generator', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        #predictions = tf.nn.softmax(logits)

        # return actions logits
        return logits

In [15]:
# Discriminator/Dopamine: Reward function/planner/naviator/advisor/supervisor/cortical columns
def discriminator(states, actions, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('discriminator', reuse=reuse):
        # Fusion/merge states and actions/ SA/ SM
        x_fused = tf.concat(axis=1, values=[states, actions])
        
        # First fully connected layer
        h1 = tf.layers.dense(inputs=x_fused, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=1)        
        #predictions = tf.nn.softmax(logits)

        # return rewards logits
        return logits

In [16]:
def model_loss(action_size, hidden_size, states, actions, targetQs, reward):
    actions_logits = generator(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    neg_log_prob_actions = tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels)
    
    Qs_logits = discriminator(actions=actions_logits, hidden_size=hidden_size, states=states)
    d_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs_logits,
                                                                    labels=reward*tf.ones_like(Qs_logits)))
    g_lossQ = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qs_logits[:-1],
                                                                     labels=tf.reshape(tf.nn.sigmoid(targetQs[1:]), 
                                                                         shape=[-1, 1])))
    #g_lossP = tf.reduce_mean(neg_log_prob_actions * tf.nn.sigmoid(targetQs))
    #g_lossP = tf.reduce_mean(neg_log_prob_actions * targetQs)
    g_lossP = tf.reduce_mean(neg_log_prob_actions[:-1] * targetQs[1:])
    #g_loss = g_lossQ + g_lossP
    g_loss = g_lossP
    d_loss += g_lossQ

    return actions_logits, Qs_logits, g_loss, d_loss, g_lossQ, g_lossP

In [17]:
# Optimizating/training/learning G & D
def model_opt(g_loss, d_loss, learning_rate):
    """
    Get optimization operations in order
    :param g_loss: Generator loss Tensor for action prediction
    :param d_loss: Discriminator loss Tensor for reward prediction for generated/prob/logits action
    :param learning_rate: Learning Rate Placeholder
    :return: A tuple of (qfunction training, generator training, discriminator training)
    """
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    g_vars = [var for var in t_vars if var.name.startswith('generator')]
    d_vars = [var for var in t_vars if var.name.startswith('discriminator')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        g_opt = tf.train.AdamOptimizer(learning_rate).minimize(g_loss, var_list=g_vars)
        d_opt = tf.train.AdamOptimizer(learning_rate).minimize(d_loss, var_list=d_vars)

    return g_opt, d_opt

In [18]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, learning_rate):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.targetQs, self.reward = model_input(state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.Qs_logits, self.g_loss, self.d_loss, self.g_lossQ, self.g_lossP = model_loss(
            action_size=action_size, hidden_size=hidden_size, # model init parameters
            states=self.states, actions=self.actions, # model input
            targetQs=self.targetQs, reward=self.reward) # model input
        
        # Update the model: backward pass and backprop
        self.g_opt, self.d_opt = model_opt(g_loss=self.g_loss, d_loss=self.d_loss, learning_rate=learning_rate)

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [19]:
print('state size:{}'.format(states.shape), 
      'actions:{}'.format(actions.shape)) 
print('action size:{}'.format(np.max(actions) - np.min(actions)+1))

state size:(1111, 4) actions:(1111,)
action size:2


In [20]:
# Training parameters
# Network parameters
state_size = 4               # number of units for the input state/observation -- simulation
action_size = 2              # number of units for the output actions -- simulation
hidden_size = 64             # number of units in each Q-network hidden layer -- simulation
learning_rate = 0.001          # learning rate for adam

In [21]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, hidden_size=hidden_size, state_size=state_size, learning_rate=learning_rate)

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [22]:
# import gym

# ## Create the Cart-Pole game environment
# env = gym.make('CartPole-v0')
# env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# # env = gym.make('MountainCar-v0')
# # env = gym.make('Pendulum-v0')
# # env = gym.make('Blackjack-v0')
# # env = gym.make('FrozenLake-v0')
# # env = gym.make('AirRaid-ram-v0')
# # env = gym.make('AirRaid-v0')
# # env = gym.make('BipedalWalker-v2')
# # env = gym.make('Copy-v0')
# # env = gym.make('CarRacing-v0')
# # env = gym.make('Ant-v2') #mujoco
# # env = gym.make('FetchPickAndPlace-v1') # mujoco required!

In [ ]:
from collections import deque
episodes_total_reward = deque(maxlen=100) # 100 episodes average/running average/running mean/window
saver = tf.train.Saver()
rewards_list, g_loss_list, d_loss_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Training episodes/epochs
    for ep in range(11111):
        batch = [] # every data batch
        total_reward = 0
        state = env.reset() # env first state

        # Training steps/batches
        while True:
            action_logits, Q_logits = sess.run(fetches=[model.actions_logits, model.Qs_logits], 
                                               feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            batch.append([state, action, Q_logits])
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done is True: # episode ended success/failure
                episodes_total_reward.append(total_reward) # stopping criteria
                rate = total_reward/ 500 # success is 500 points
                break

        # Training using batches
        #batch = memory.buffer
        states = np.array([each[0] for each in batch])
        actions = np.array([each[1] for each in batch])
        targetQs = np.array([each[2] for each in batch])
        g_lossQ, g_lossP, g_loss, d_loss, _, _ = sess.run([model.g_lossQ, model.g_lossP, 
                                                           model.g_loss, model.d_loss, 
                                                           model.g_opt, model.d_opt],
                                                          feed_dict = {model.states: states, 
                                                                       model.actions: actions,
                                                                       model.reward: rate,
                                                                       model.targetQs: targetQs.reshape([-1])})
        # Average 100 episode total reward
        # Print out
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episodes_total_reward)),
              'glossQ:{:.4f}'.format(g_lossQ),
              'glossP:{:.4f}'.format(g_lossP),
              'gloss:{:.4f}'.format(g_loss),
              'dloss:{:.4f}'.format(d_loss))
        # Ploting out
        rewards_list.append([ep, np.mean(episodes_total_reward)])
        g_loss_list.append([ep, g_loss])
        d_loss_list.append([ep, d_loss])
        # Break episode/epoch loop
        if np.mean(episodes_total_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:52.0000 glossQ:0.6928 glossP:0.0546 gloss:0.0546 dloss:1.4188
Episode:1 meanR:49.5000 glossQ:0.6932 glossP:0.0358 gloss:0.0358 dloss:1.4081
Episode:2 meanR:41.0000 glossQ:0.6925 glossP:0.0600 gloss:0.0600 dloss:1.4244
Episode:3 meanR:36.2500 glossQ:0.6931 glossP:0.0349 gloss:0.0349 dloss:1.4080
Episode:4 meanR:31.0000 glossQ:0.6923 glossP:0.0578 gloss:0.0578 dloss:1.4197
Episode:5 meanR:27.5000 glossQ:0.6920 glossP:0.0445 gloss:0.0445 dloss:1.4104
Episode:6 meanR:25.1429 glossQ:0.6926 glossP:-0.0029 gloss:-0.0029 dloss:1.3799
Episode:7 meanR:23.1250 glossQ:0.6917 glossP:-0.0613 gloss:-0.0613 dloss:1.3349
Episode:8 meanR:21.5556 glossQ:0.6890 glossP:-0.1059 gloss:-0.1059 dloss:1.3006
Episode:9 meanR:20.5000 glossQ:0.6842 glossP:-0.1593 gloss:-0.1593 dloss:1.2629
Episode:10 meanR:19.5455 glossQ:0.6785 glossP:-0.2131 gloss:-0.2131 dloss:1.2295
Episode:11 meanR:18.8333 glossQ:0.6719 glossP:-0.2757 gloss:-0.2757 dloss:1.1974
Episode:12 meanR:18.6154 glossQ:0.6795 glossP:-0.2

Episode:102 meanR:40.5700 glossQ:0.3421 glossP:-1.4682 gloss:-1.4682 dloss:1.8704
Episode:103 meanR:42.1300 glossQ:0.3621 glossP:-1.4007 gloss:-1.4007 dloss:1.2129
Episode:104 meanR:43.1000 glossQ:0.3413 glossP:-1.4748 gloss:-1.4748 dloss:0.9163
Episode:105 meanR:43.1300 glossQ:0.3372 glossP:-1.4885 gloss:-1.4885 dloss:0.5093
Episode:106 meanR:43.1400 glossQ:0.3056 glossP:-1.6062 gloss:-1.6062 dloss:0.4652
Episode:107 meanR:43.1500 glossQ:0.2955 glossP:-1.6554 gloss:-1.6554 dloss:0.4469
Episode:108 meanR:43.1700 glossQ:0.2855 glossP:-1.6976 gloss:-1.6976 dloss:0.4386
Episode:109 meanR:43.1700 glossQ:0.3028 glossP:-1.6277 gloss:-1.6277 dloss:0.4613
Episode:110 meanR:43.6400 glossQ:0.4126 glossP:-1.2324 gloss:-1.2324 dloss:0.7751
Episode:111 meanR:48.5300 glossQ:0.4568 glossP:-1.0940 gloss:-1.0940 dloss:2.2339
Episode:112 meanR:49.1400 glossQ:0.4191 glossP:-1.2226 gloss:-1.2226 dloss:0.8590
Episode:113 meanR:50.2000 glossQ:0.4444 glossP:-1.1351 gloss:-1.1351 dloss:1.0440
Episode:114 mean

Episode:207 meanR:33.3800 glossQ:0.1980 glossP:-2.0377 gloss:-2.0377 dloss:0.6511
Episode:208 meanR:33.6200 glossQ:0.2020 glossP:-2.0086 gloss:-2.0086 dloss:0.4592
Episode:209 meanR:33.9800 glossQ:0.2153 glossP:-1.9396 gloss:-1.9396 dloss:0.5388
Episode:210 meanR:33.8200 glossQ:0.2150 glossP:-1.9407 gloss:-1.9407 dloss:0.5046
Episode:211 meanR:29.1100 glossQ:0.2163 glossP:-1.9365 gloss:-1.9365 dloss:0.4379
Episode:212 meanR:28.7100 glossQ:0.2205 glossP:-1.9182 gloss:-1.9182 dloss:0.4868
Episode:213 meanR:27.7700 glossQ:0.2235 glossP:-1.9030 gloss:-1.9030 dloss:0.4673
Episode:214 meanR:27.6500 glossQ:0.2222 glossP:-1.9100 gloss:-1.9100 dloss:0.4050
Episode:215 meanR:27.6800 glossQ:0.2333 glossP:-1.8590 gloss:-1.8590 dloss:0.5257
Episode:216 meanR:27.5700 glossQ:0.2199 glossP:-1.9280 gloss:-1.9280 dloss:0.4362
Episode:217 meanR:27.4600 glossQ:0.2271 glossP:-1.8887 gloss:-1.8887 dloss:0.4380
Episode:218 meanR:27.2400 glossQ:0.2041 glossP:-2.0064 gloss:-2.0064 dloss:0.3454
Episode:219 mean

Episode:308 meanR:59.7800 glossQ:0.4041 glossP:-1.3433 gloss:-1.3433 dloss:0.7226
Episode:309 meanR:59.8400 glossQ:0.5118 glossP:-0.9325 gloss:-0.9325 dloss:0.8873
Episode:310 meanR:59.8500 glossQ:0.5180 glossP:-0.9106 gloss:-0.9106 dloss:0.8675
Episode:311 meanR:59.8000 glossQ:0.5259 glossP:-0.8800 gloss:-0.8800 dloss:0.8337
Episode:312 meanR:59.6800 glossQ:0.5188 glossP:-0.9021 gloss:-0.9021 dloss:0.8240
Episode:313 meanR:59.5600 glossQ:0.5145 glossP:-0.9163 gloss:-0.9163 dloss:0.8059
Episode:314 meanR:59.5100 glossQ:0.5137 glossP:-0.9194 gloss:-0.9194 dloss:0.7942
Episode:315 meanR:59.2700 glossQ:0.5049 glossP:-0.9479 gloss:-0.9479 dloss:0.7812
Episode:316 meanR:59.1900 glossQ:0.4946 glossP:-0.9795 gloss:-0.9795 dloss:0.7690
Episode:317 meanR:59.0700 glossQ:0.4868 glossP:-1.0043 gloss:-1.0043 dloss:0.7421
Episode:318 meanR:59.0700 glossQ:0.4764 glossP:-1.0371 gloss:-1.0371 dloss:0.7244
Episode:319 meanR:59.1200 glossQ:0.4654 glossP:-1.0714 gloss:-1.0714 dloss:0.7097
Episode:320 mean

Episode:413 meanR:34.1300 glossQ:0.2688 glossP:-1.7428 gloss:-1.7428 dloss:0.4687
Episode:414 meanR:34.2100 glossQ:0.2615 glossP:-1.7848 gloss:-1.7848 dloss:0.4676
Episode:415 meanR:34.1700 glossQ:0.2069 glossP:-2.0310 gloss:-2.0310 dloss:0.3457
Episode:416 meanR:34.1600 glossQ:0.2427 glossP:-1.8664 gloss:-1.8664 dloss:0.4148
Episode:417 meanR:34.1500 glossQ:0.2382 glossP:-1.8884 gloss:-1.8884 dloss:0.3828
Episode:418 meanR:34.9200 glossQ:0.2613 glossP:-1.7681 gloss:-1.7681 dloss:0.8066
Episode:419 meanR:35.3100 glossQ:0.2411 glossP:-1.8664 gloss:-1.8664 dloss:0.6056
Episode:420 meanR:35.4000 glossQ:0.2529 glossP:-1.8079 gloss:-1.8079 dloss:0.4347
Episode:421 meanR:35.4000 glossQ:0.1968 glossP:-2.0821 gloss:-2.0821 dloss:0.3627
Episode:422 meanR:35.4000 glossQ:0.1624 glossP:-2.2793 gloss:-2.2793 dloss:0.2894
Episode:423 meanR:35.4300 glossQ:0.1705 glossP:-2.2190 gloss:-2.2190 dloss:0.3161
Episode:424 meanR:35.4300 glossQ:0.1642 glossP:-2.2605 gloss:-2.2605 dloss:0.2785
Episode:425 mean

...

Episode:2321 meanR:135.7200 glossQ:0.4542 glossP:-1.1431 gloss:-1.1431 dloss:0.7132
Episode:2322 meanR:135.1100 glossQ:0.4676 glossP:-1.1007 gloss:-1.1007 dloss:0.7366
Episode:2323 meanR:134.9300 glossQ:0.4536 glossP:-1.1462 gloss:-1.1462 dloss:0.7197
Episode:2324 meanR:134.7100 glossQ:0.4715 glossP:-1.0933 gloss:-1.0933 dloss:0.7552
Episode:2325 meanR:134.3600 glossQ:0.4727 glossP:-1.0895 gloss:-1.0895 dloss:0.7514
Episode:2326 meanR:134.2400 glossQ:0.4882 glossP:-1.0383 gloss:-1.0383 dloss:0.7908
Episode:2327 meanR:134.2000 glossQ:0.4890 glossP:-1.0334 gloss:-1.0334 dloss:0.8228
Episode:2328 meanR:134.0800 glossQ:0.4593 glossP:-1.1242 gloss:-1.1242 dloss:0.8446
Episode:2329 meanR:134.1600 glossQ:0.4106 glossP:-1.2790 gloss:-1.2790 dloss:0.8637
Episode:2330 meanR:134.9000 glossQ:0.5149 glossP:-0.9203 gloss:-0.9203 dloss:1.0700
Episode:2331 meanR:135.9900 glossQ:0.5898 glossP:-0.6597 gloss:-0.6597 dloss:1.2723
Episode:2332 meanR:140.1800 glossQ:0.6582 glossP:-0.4053 gloss:-0.4053 dloss

Episode:2419 meanR:197.9700 glossQ:0.6819 glossP:-0.1437 gloss:-0.1437 dloss:1.4164
Episode:2420 meanR:202.7400 glossQ:0.6944 glossP:-0.0282 gloss:-0.0282 dloss:1.4088
Episode:2421 meanR:207.5800 glossQ:0.6962 glossP:-0.0137 gloss:-0.0137 dloss:1.4010
Episode:2422 meanR:210.2300 glossQ:0.6896 glossP:-0.0841 gloss:-0.0841 dloss:1.3958
Episode:2423 meanR:212.4100 glossQ:0.6897 glossP:-0.0723 gloss:-0.0723 dloss:1.3852
Episode:2424 meanR:213.1400 glossQ:0.6827 glossP:-0.1621 gloss:-0.1621 dloss:1.3156
Episode:2425 meanR:213.8000 glossQ:0.6827 glossP:-0.1563 gloss:-0.1563 dloss:1.3151
Episode:2426 meanR:214.4700 glossQ:0.6820 glossP:-0.1689 gloss:-0.1689 dloss:1.3141
Episode:2427 meanR:214.9300 glossQ:0.6812 glossP:-0.1654 gloss:-0.1654 dloss:1.3112
Episode:2428 meanR:215.2300 glossQ:0.6807 glossP:-0.1717 gloss:-0.1717 dloss:1.3089
Episode:2429 meanR:215.3100 glossQ:0.6793 glossP:-0.1843 gloss:-0.1843 dloss:1.3024
Episode:2430 meanR:215.1500 glossQ:0.6802 glossP:-0.1826 gloss:-0.1826 dloss

Episode:2518 meanR:155.6800 glossQ:0.6208 glossP:-0.5184 gloss:-0.5184 dloss:1.3471
Episode:2519 meanR:157.5500 glossQ:0.6877 glossP:-0.2074 gloss:-0.2074 dloss:1.5434
Episode:2520 meanR:155.0100 glossQ:0.6715 glossP:-0.2317 gloss:-0.2317 dloss:1.3944
Episode:2521 meanR:151.7900 glossQ:0.6477 glossP:-0.3878 gloss:-0.3878 dloss:1.3183
Episode:2522 meanR:150.6600 glossQ:0.6210 glossP:-0.5271 gloss:-0.5271 dloss:1.2717
Episode:2523 meanR:148.4400 glossQ:0.4586 glossP:-1.1237 gloss:-1.1237 dloss:0.7343
Episode:2524 meanR:147.6100 glossQ:0.3843 glossP:-1.3886 gloss:-1.3886 dloss:0.6210
Episode:2525 meanR:146.8900 glossQ:0.4151 glossP:-1.2737 gloss:-1.2737 dloss:0.6765
Episode:2526 meanR:147.4000 glossQ:0.5644 glossP:-0.7280 gloss:-0.7280 dloss:1.1868
Episode:2527 meanR:148.1800 glossQ:0.6138 glossP:-0.5619 gloss:-0.5619 dloss:1.2489
Episode:2528 meanR:149.1700 glossQ:0.6266 glossP:-0.5075 gloss:-0.5075 dloss:1.2975
Episode:2529 meanR:150.3400 glossQ:0.6401 glossP:-0.4500 gloss:-0.4500 dloss

Episode:2616 meanR:209.7700 glossQ:0.4955 glossP:-0.9763 gloss:-0.9763 dloss:0.8402
Episode:2617 meanR:211.3700 glossQ:0.5994 glossP:-0.5829 gloss:-0.5829 dloss:1.2724
Episode:2618 meanR:209.4000 glossQ:0.4342 glossP:-1.1965 gloss:-1.1965 dloss:0.7562
Episode:2619 meanR:204.5300 glossQ:0.4289 glossP:-1.2056 gloss:-1.2056 dloss:0.7379
Episode:2620 meanR:203.1000 glossQ:0.6363 glossP:-0.4299 gloss:-0.4299 dloss:1.2188
Episode:2621 meanR:203.0000 glossQ:0.6554 glossP:-0.2722 gloss:-0.2722 dloss:1.3335
Episode:2622 meanR:203.4200 glossQ:0.6770 glossP:-0.0786 gloss:-0.0786 dloss:1.3853
Episode:2623 meanR:206.7000 glossQ:0.6831 glossP:0.0446 gloss:0.0446 dloss:1.3813
Episode:2624 meanR:211.6000 glossQ:0.6981 glossP:0.0605 gloss:0.0605 dloss:1.3506
Episode:2625 meanR:214.7400 glossQ:0.6858 glossP:-0.0466 gloss:-0.0466 dloss:1.4022
Episode:2626 meanR:214.4900 glossQ:0.6715 glossP:-0.2260 gloss:-0.2260 dloss:1.3104
Episode:2627 meanR:213.7200 glossQ:0.6607 glossP:-0.3352 gloss:-0.3352 dloss:1.2

Episode:2714 meanR:138.9500 glossQ:0.6794 glossP:-0.2243 gloss:-0.2243 dloss:1.4076
Episode:2715 meanR:140.5300 glossQ:0.6841 glossP:-0.1981 gloss:-0.1981 dloss:1.4469
Episode:2716 meanR:145.3800 glossQ:0.6981 glossP:-0.1594 gloss:-0.1594 dloss:1.5207
Episode:2717 meanR:148.6700 glossQ:0.6924 glossP:-0.2830 gloss:-0.2830 dloss:1.6179
Episode:2718 meanR:152.9300 glossQ:0.6392 glossP:-0.4449 gloss:-0.4449 dloss:1.6585
Episode:2719 meanR:155.5300 glossQ:0.6306 glossP:-0.4884 gloss:-0.4884 dloss:1.4337
Episode:2720 meanR:156.1600 glossQ:0.5807 glossP:-0.7021 gloss:-0.7021 dloss:1.2408
Episode:2721 meanR:155.6600 glossQ:0.5353 glossP:-0.8599 gloss:-0.8599 dloss:1.0989
Episode:2722 meanR:154.1000 glossQ:0.4037 glossP:-1.2975 gloss:-1.2975 dloss:0.7743
Episode:2723 meanR:150.8600 glossQ:0.4682 glossP:-1.1015 gloss:-1.1015 dloss:0.7599
Episode:2724 meanR:145.9700 glossQ:0.3880 glossP:-1.3690 gloss:-1.3690 dloss:0.6260
Episode:2725 meanR:142.8200 glossQ:0.3775 glossP:-1.4135 gloss:-1.4135 dloss

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(g_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('G losses')

In [ ]:
eps, arr = np.array(d_loss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('D losses')

## Testing

Let's checkout how our trained agent plays the game.

In [28]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')
# env = gym.make('Acrobot-v1')
# env = gym.make('MountainCar-v0')
# env = gym.make('Pendulum-v0')
# env = gym.make('Blackjack-v0')
# env = gym.make('FrozenLake-v0')
# env = gym.make('AirRaid-ram-v0')
# env = gym.make('AirRaid-v0')
# env = gym.make('BipedalWalker-v2')
# env = gym.make('Copy-v0')
# env = gym.make('CarRacing-v0')
# env = gym.make('Ant-v2') #mujoco
# env = gym.make('FetchPickAndPlace-v1') # mujoco required!

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(1):
    #while True:
        state = env.reset()
        total_reward = 0

        # Steps/batches
        #for _ in range(111111111111111111):
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: np.reshape(state, [1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                break
        # Print and break condition
        print('total_reward: {}'.format(total_reward))
        if total_reward == 500:
            break
                
# Closing the env
env.close()

/home/arasdar/anaconda3/envs/env/lib/python3.6/site-packages/gym/__init__.py:22: UserWarning: DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.
  warnings.warn('DEPRECATION WARNING: to improve load times, gym no longer automatically loads gym.spaces. Please run "import gym.spaces" to load gym.spaces on your own. This warning will turn into an error in a future version of gym.')


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from checkpoints/model2.ckpt
total_reward: 500.0


## Extending this

So, Cart-Pole is a pretty simple game. However, the same model can be used to train an agent to play something much more complicated like Pong or Space Invaders. Instead of a state like we're using here though, you'd want to use convolutional layers to get the state from the screen images.

![Deep Q-Learning Atari](assets/atari-network.png)

I'll leave it as a challenge for you to use deep Q-learning to train an agent to play Atari games. Here's the original paper which will get you started: http://www.davidqiu.com:8888/research/nature14236.pdf.